# Build a Simple LLM Applicatin with LCEL

- LCEL : LangChain Expression Language
- used to chain components together.

In [3]:
# Load environment variables and create LLAMA3.1 model instance
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [4]:
# ChatModels are instances of Langchain runnables meaning they expose a standard interface to interact with them
# Provide context to the LLM which can be a list of past conversation between LLM and user and LLM will generate 
# output keeping past conversation in mind

# SystemMessage: This message provides instructions or context to the LLM. It's often used to set the tone of the conversation.
# HumanMessage: This represents a message from the user. It's the input to the LLM.
# AIMessage: This represents a message generated by the LLM in response to a HumanMessage. 
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

c:\Users\RishilRamesh\My-Courses\LangChain\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:381: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content='ciao!', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-49c30fb7-f534-42db-8041-df9459e847b8-0', usage_metadata={'input_tokens': 10, 'output_tokens': 2, 'total_tokens': 12})

In [5]:
# LLMs response is an AIMessage object, when we want to work with plain strings, we can use below parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

c:\Users\RishilRamesh\My-Courses\LangChain\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:381: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Ciao!'

In [6]:
# More commonly, we can "chain" the model with this output parser. Output of left component in chain is fed as input to the next component.
# This means this output parser will get called every time in this chain. 
# This chain takes on the input type of the language model (string or list of message) and 
# returns the output type of the output parser (string).

chain = model | parser
chain.invoke(messages)

c:\Users\RishilRamesh\My-Courses\LangChain\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:381: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'ciao!'

In [7]:
# The ChatPromptTemplate in LangChain is a way to create structured conversation prompts for chatbots or conversational agents.
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [8]:
# Read the messages as list
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [9]:
# Now chain together the prompt template with LLM and parser
chain = prompt_template | model | parser
chain.invoke({"language": "japanese", "text": "hi"})

c:\Users\RishilRamesh\My-Courses\LangChain\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:381: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'こんにちは'

In [10]:
# Run this code if we have the serve.py running where we wrote the chain to interact with LLM
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'ciao'